# 모듈만들기

## 신규회원 정보가져오기

In [7]:
def nextTime(x,y):
    time1 = x.split('-')
    time1 = datetime(int(time1[0]), int(time1[1]), int(time1[2]))
    return time1 + timedelta(days=(y-1))

def getOrder(userId):
    try:
        url="https://secure.iherb.com/myaccount/orders"
        driver = wb.Chrome()
        driver.get(url)

        cursor.execute("""select * from mbr where id='{}'""".format(userId))
        list1 = cursor.fetchone()
        # iherb id,pw db에서 가져오기
        user_id = list1[2]
        user_pw = list1[3]

        iherbId = driver.find_element_by_id("username_input")
        iherbPw = driver.find_element_by_id("Password")

        iherbId.send_keys(user_id)
        iherbPw.send_keys(user_pw)
        iherbPw.send_keys(Keys.ENTER)

        try:
            date = driver.find_element_by_class_name("custom-dropdown")
            date.click()
            recent = driver.find_element_by_css_selector("ul#date > li:nth-child(6)")
            recent.click()
        except:
            print('정보가 없습니다.')

        try:
            date = driver.find_element_by_class_name("custom-dropdown")
            date.click()
            recent = driver.find_element_by_css_selector("ul#date > li:nth-child(2)")
            recent.click()
        except:
            print("정보가 없습니다.")

        months = {"Janyary":1,"February":2,"March":3,"April":4,"May":5,"June":6,"July":7,"August":8,"September":9,"October":10,"November":11,"December":12}

        date_list = []
        order_num = []
        order_date = driver.find_elements_by_css_selector("div.order-date > span.val")
        order_list = driver.find_elements_by_css_selector("div.order-number strong")

        for i in range(len(order_date)):
            date = order_date[i].text.strip().split(" ")
            date[0] = int(months[date[0]])
            date[1] = int(date[1])
            date[2] = int(date[2])
            date = "{}-{}-{}".format(date[2],date[0],date[1])
            date_list.append(date)
            order_num.append(order_list[i].text.strip())
        # 오늘 날짜 구하기 YYYY-MM-DD
        today = time.strftime('%Y-%m-%d', time.localtime(time.time()))

        code_list = []
        code = []
        url = driver.current_url

        for i in range(1,len(order_date)+1):
            item = driver.find_elements_by_css_selector("#OrderHistoryResults > div.col-xs-24.order-history-results.module > article:nth-child({}) div.products a.desc".format(i))
            print(len(item))

            for j in range(0,len(item)):
                try:
                    item = driver.find_elements_by_css_selector("#OrderHistoryResults > div.col-xs-24.order-history-results.module > article:nth-child({}) div.products a.desc".format(i))
                    item[j].click()
                    time.sleep(2)
                    code.append(driver.find_element_by_xpath("//*[@id='product-specs-list']/li[3]/span").text.strip())
                    driver.get(url)
                except:
                    print("제품이 없음")
                    driver.get(url)
                    time.sleep(2)
            code_list.append(code)
            code = []

        next_date_list = []
        for i in range(len(order_date)):
            for j in range(len(code_list[i])):
                cursor.execute("""select * from drug where pcode='{}'""".format(code_list[i][j]))
                next_date = cursor.fetchone()
                next_date_list.append(nextTime(date_list[i],next_date[4]))
        for i in range(len(order_num)):
            for j in range(len(code_list[i])):
                next_date = "{}-{}-{}".format(
                    next_date_list[i].year,
                next_date_list[i].month,
                next_date_list[i].day
                )

                cursor.execute("""insert into ordered values({},'{}','{}',to_date('{}','yyyy-mm-dd hh24:mi:ss'),to_date('{}','yyyy-mm-dd hh24:mi:ss'))""".format(order_num[i],userId,code_list[i][j],date_list[i],next_date))
                connection.commit()

        print("소비내역 넣는 크롤링끝")
    except Exception  as ex:
        print("Error발생.....:"+ex)